# Задание 1 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [2]:
import pandas as pd

In [5]:
ratings = pd.read_csv(r'C:\Users\AsusTeck\Desktop\ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [21]:
ratings_grouped = ratings.groupby('userId').count().reset_index()
ratings_filtered = ratings_grouped[ratings_grouped['rating']>=100]
film_fans_user_ids = ratings_filtered['userId'].tolist()
hundred = ratings[ ratings['userId'].isin(film_fans_user_ids) ]
hundred.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [19]:
mean_time = hundred.groupby('userId').mean()
mean_time.head()

,movieId,rating,timestamp
userId,,,
4,1721.514706,4.348039,9.498806e+08
5,6655.940000,3.910000,1.163374e+09
8,6383.620690,3.866379,1.154438e+09
15,29364.931765,2.621765,1.187732e+09
17,4893.936639,3.743802,1.127471e+09


#  Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

1) таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
2) аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [12]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd = rzd[['client_id', 'rzd_revenue']]
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [13]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto = auto[['client_id', 'auto_revenue']]
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [14]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air = air[['client_id', 'air_revenue']]
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [15]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base = client_base[['client_id', 'address']]
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [39]:
from functools import reduce

In [41]:
df1 = [rzd, auto, air]
without_address = reduce(lambda  left,right: pd.merge(left,right,on=['client_id'],
                                            how='outer'), df1)
without_address

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [42]:
with_address = without_address.merge(client_base, on='client_id',
                    how='outer')
with_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


#  Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

На примере датафрейма, который я выдернула из занятия. Создадим примерный образец датафрейма с координатами.

In [63]:
address = pd.DataFrame(
    {
        'user_id': [11, 11, 22, 55, 77],
        'latitude': [815, 8769, 4, 13, 1],
        'longitude': [78493,4003,53,1, 1]
    }
)
x = address.groupby('user_id').apply(lambda x: x.sort_values('latitude'))
x = x[['latitude', 'longitude']]
x

latitude  longitude
user_id                       
11      0       815      78493
        1      8769       4003
22      2         4         53
55      3        13          1
77      4         1          1

In [ ]:
y = a.merge(x, on='user_id', how='outer')
y = y.groupby('user_id').apply(lambda x: x.sort_values('visits'))
y

Так как весь код занятия я переносить сюда не стала (а тестировала я систему в ноуте занятия), то уточню, что a - это a = visits_grouped.join(purchases_pivot). В идеале можно еще перезаписать датафрейм без столбца user_id на выходе, но мне уже было лень) Если надо, то скрин итоговых данных могу скинуть в slack.